In [1]:
from mongodbcredentials import CONNECTION_STRING
from pymongo import MongoClient
import certifi
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from twitter_modules import get_mostcommon_posts, save_results, database_as_dictionary, pos_neg_count, plot_topics

In [2]:
client = MongoClient(CONNECTION_STRING, tlsCAFile=certifi.where())

In [3]:
twitter_facemasks = client.TwitterFacemasks
twitter_lockdown = client.TwitterLockdown
twitter_pcr = client.TwitterPCR
twitter_pfizer = client.TwitterPfizer
twitter_quarantine = client.TwitterQuarantine
twitter_restrictions = client.TwitterRestrictions
twitter_vaccine = client.TwitterVaccination

In [4]:
analyzer = SentimentIntensityAnalyzer()

In [5]:
facemasks_dict = database_as_dictionary(twitter_facemasks, analyzer)
lockdown_dict = database_as_dictionary(twitter_lockdown, analyzer)
pcr_dict = database_as_dictionary(twitter_pcr, analyzer)
pfizer_dict = database_as_dictionary(twitter_pfizer, analyzer)
quarantine_dict = database_as_dictionary(twitter_quarantine, analyzer)
restrictions_dict = database_as_dictionary(twitter_restrictions, analyzer)
vaccine_dict = database_as_dictionary(twitter_vaccine, analyzer)

In [ ]:
import pandas as pd

In [ ]:
facemasks_df = pd.DataFrame(facemasks_dict, columns=['tweet', 'sentiment'])
lockdown_df = pd.DataFrame(lockdown_dict, columns=['tweet', 'sentiment'])
pcr_df = pd.DataFrame(pcr_dict, columns=['tweet', 'sentiment'])
pfizer_df = pd.DataFrame(pfizer_dict, columns=['tweet', 'sentiment'])
quarantine_df = pd.DataFrame(quarantine_dict, columns=['tweet', 'sentiment'])
restrictions_df = pd.DataFrame(restrictions_dict, columns=['tweet', 'sentiment'])
vaccine_df = pd.DataFrame(vaccine_dict, columns=['tweet', 'sentiment'])

In [ ]:
lockdown_df["tweet"].nunique()

137365

In [ ]:
lockdown_df

,tweet,sentiment
0,Any form of lockdown will be a death sentence ...,negative
1,I was going to renew my passport but everytime...,neutral
2,I don't usually agree with anything Julia Hart...,positive
3,It’s fascinating how people run to this space ...,positive
4,@SupDog_SkaDog apparently there's a new varian...,negative
...,...,...
137360,COVID in Germany: Bavaria cancels Christmas ma...,negative
137361,USDCAD: Oil Drop &amp; European Lockdown Risks...,negative
137362,There can be no doubt: the EU &amp; constituti...,positive
137363,Austria becomes the first European country to ...,negative


In [ ]:
positives = lockdown_df[lockdown_df["sentiment"] == "positive"]
negatives = lockdown_df[lockdown_df["sentiment"] == "negative"]
neutrals = lockdown_df[lockdown_df["sentiment"] == "neutral"]

,tweet,sentiment
1,I was going to renew my passport but everytime...,neutral
6,@NoWayHomeShill Boris when he puts England int...,neutral
14,I can't handle another lockdown.,neutral
15,"Commentary - That's right, blues fans, Eric Cl...",neutral
16,Bojo wont announce the firebreak lockdown toda...,neutral
...,...,...
137347,"""Austria announces a lockdown and vaccination ...",neutral
137348,"""Austria announces a lockdown and vaccination ...",neutral
137352,we're on official ryujin lockdown,neutral
137353,@FlimsRadar @EmmaAgyemang @JimHarraHMRC Jim's ...,neutral


In [ ]:
!pip install transformers
!pip install datasets

In [10]:
from transformers import pipeline
#SentimentClassifier = pipeline("sentiment-analysis")
TweetBert = pipeline(model="TweetBERT")

RuntimeError: Instantiating a pipeline without a task set raised an error: 404 Client Error: Not Found for url: https://huggingface.co/api/models/TweetBERT

In [9]:
SentimentClassifier(["I am getting the vaccine"])

[{'label': 'NEGATIVE', 'score': 0.7611358761787415}]

In [16]:
from datasets import load_dataset

In [19]:
dataset = load_dataset('csv', data_files={'train': 'Corona_NLP_train.csv', 'test': 'Corona_NLP_test.csv'}, encoding = "ISO-8859-1")

Using custom data configuration default-e7940792e5b71163
Reusing dataset csv (C:\Users\craig\.cache\huggingface\datasets\csv\default-e7940792e5b71163\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
100%|██████████| 2/2 [00:00<00:00, 48.86it/s]


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

Downloading: 100%|██████████| 747/747 [00:00<00:00, 373kB/s]
Downloading: 100%|██████████| 878k/878k [00:00<00:00, 1.65MB/s]
Downloading: 100%|██████████| 446k/446k [00:00<00:00, 815kB/s] 
Downloading: 100%|██████████| 150/150 [00:00<00:00, 99.9kB/s]
Downloading: 100%|██████████| 476M/476M [01:48<00:00, 4.58MB/s] 


In [13]:
def transform_labels(label):

    label = label['Sentiment']
    num = 0
    if label == 'Positive' or label == 'Extremely Positive':
        num = 0
    elif label == 'Negative' or label == 'Extremely Negative':
        num = 1
    elif label == 'Neutral':
        num = 2

    return {'labels': num}

In [14]:
def tokenize_data(example):
    return tokenizer(example['OriginalTweet'], padding='max_length')

In [17]:
!pip install datasets

In [20]:
dataset = dataset.map(tokenize_data, batched=True)

remove_columns = ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

100%|██████████| 4/4 [00:00<00:00,  6.36ba/s]
41157ex [00:06, 6224.91ex/s]
3798ex [00:00, 6171.05ex/s]


In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [36]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model.save_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
tokenizer.save_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

loading configuration file cardiffnlp/twitter-roberta-base-sentiment\config.json
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_siz

('cardiffnlp/twitter-roberta-base-sentiment\\tokenizer_config.json',
 'cardiffnlp/twitter-roberta-base-sentiment\\special_tokens_map.json',
 'cardiffnlp/twitter-roberta-base-sentiment\\vocab.json',
 'cardiffnlp/twitter-roberta-base-sentiment\\merges.txt',
 'cardiffnlp/twitter-roberta-base-sentiment\\added_tokens.json',
 'cardiffnlp/twitter-roberta-base-sentiment\\tokenizer.json')

In [39]:
from scipy.special import softmax

In [69]:
text = "I will be getting the vaccine for covid-19"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(output)
scores = output[0][0].detach().numpy()
print(scores)
scores = softmax(scores)
print(scores)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.1705,  1.8912, -0.8896]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
[-1.1704623  1.8912349 -0.8895886]
[0.04221534 0.90187943 0.05590519]


In [50]:
import urllib.request
import csv
import numpy as np

In [48]:
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [53]:
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [70]:
print(ranking)

[0 1 2]


In [71]:
ranking = np.argsort(scores)
ranking = ranking[::-1]

#for i in range(scores.shape[0]):
l = labels[ranking[0]]
print(l)
   # s = scores[ranking[i]]
    #print(f"{i+1}) {l} {np.round(float(s), 4)}")

neutral


In [29]:
train_dataset = dataset['train'].shuffle(seed=10).select(range(40000))
eval_dataset = dataset['train'].shuffle(seed=10).select(range(40000, 41000))

Loading cached shuffled indices for dataset at C:\Users\craig\.cache\huggingface\datasets\csv\default-e7940792e5b71163\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-240ce6069de3ba53.arrow
Loading cached shuffled indices for dataset at C:\Users\craig\.cache\huggingface\datasets\csv\default-e7940792e5b71163\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e\cache-240ce6069de3ba53.arrow


In [33]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [34]:
trainer.train()

***** Running training *****
  Num examples = 40000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15000



ValueError: expected sequence of length 53 at dim 1 (got 91)

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer.evaluate()